In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [58]:
data = pd.read_csv('/content/spotify_songs.csv')

#print(data)
#u_data.head()
#u_data.set_axis(data_labels, axis='columns', inplace=True) #postavljanje naziva kolona


In [59]:
print(data.columns)
with pd.option_context("display.max_rows", None, 'display.max_columns', None):
    display(data.dtypes)

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')


track_id                     object
track_name                   object
track_artist                 object
track_popularity              int64
track_album_id               object
track_album_name             object
track_album_release_date     object
playlist_name                object
playlist_id                  object
playlist_genre               object
playlist_subgenre            object
danceability                float64
energy                      float64
key                           int64
loudness                    float64
mode                          int64
speechiness                 float64
acousticness                float64
instrumentalness            float64
liveness                    float64
valence                     float64
tempo                       float64
duration_ms                   int64
dtype: object

In [71]:
# Filter rows where track artist is 'The Weeknd'
filtered_data = data[data['track_artist'] == 'The Weeknd']

# Keep only the desired columns
selected_columns = ['track_name', 'track_popularity', 'danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness',
                     'liveness', 'valence', 'tempo', 'duration_ms']

filtered_data = filtered_data[selected_columns]

# Remove duplicates, keeping the row with the highest popularity
filtered_data = filtered_data.sort_values('track_popularity', ascending=False).drop_duplicates('track_name')


filtered_data['track_popularity'] = filtered_data['track_popularity'].astype(float)
filtered_data['duration_mins'] = filtered_data['duration_ms'] / 60000.0
filtered_data = filtered_data.drop('duration_ms', axis=1)
filtered_data = filtered_data.sort_values('track_name', ascending=True)

filtered_data['track_popularity_color'] = filtered_data['track_popularity'] / 100.0

duration_min_max = filtered_data['duration_mins'].max()
filtered_data['duration_mins_color'] = filtered_data['duration_mins'] / duration_min_max

tempo_min_max = filtered_data['tempo'].max()
filtered_data['tempo_color'] = filtered_data['tempo'] / tempo_min_max


In [72]:
#renaming
filtered_data.rename(columns={'danceability': 'Danceability'}, inplace=True)
filtered_data.rename(columns={'track_popularity': 'Popularity'}, inplace=True)
filtered_data.rename(columns={'energy': 'Energy'}, inplace=True)
filtered_data.rename(columns={'speechiness': 'Speechiness'}, inplace=True)
filtered_data.rename(columns={'acousticness': 'Acousticness'}, inplace=True)
filtered_data.rename(columns={'instrumentalness': 'Instrumentalness'}, inplace=True)
filtered_data.rename(columns={'liveness': 'Liveness'}, inplace=True)
filtered_data.rename(columns={'valence': 'Valence'}, inplace=True)
filtered_data.rename(columns={'tempo': 'Tempo'}, inplace=True)
filtered_data.rename(columns={'duration_mins': 'Duration (minutes)'}, inplace=True)

In [73]:
# Print the resulting DataFrame
print(filtered_data.columns)
#print(filtered_data)

with pd.option_context("display.max_rows", None, 'display.max_columns', None):
    display(filtered_data.dtypes)

Index(['track_name', 'Popularity', 'Danceability', 'Energy', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration (minutes)', 'track_popularity_color', 'duration_mins_color',
       'tempo_color'],
      dtype='object')


track_name                 object
Popularity                float64
Danceability              float64
Energy                    float64
Speechiness               float64
Acousticness              float64
Instrumentalness          float64
Liveness                  float64
Valence                   float64
Tempo                     float64
Duration (minutes)        float64
track_popularity_color    float64
duration_mins_color       float64
tempo_color               float64
dtype: object

In [74]:
filtered_data.to_csv('filtered_data.csv', index=False)

In [76]:

melted_data = pd.melt(filtered_data, id_vars=['track_name'], value_vars=['Danceability', 'Energy', 'Speechiness', 'Acousticness', 'Instrumentalness',
                     'Liveness', 'Valence', 'Tempo', 'Duration (minutes)', 'Popularity'], var_name='Trait', value_name='value')

melted_data.columns = ['Song', 'Trait', 'value']
melted_data['Color'] = melted_data['value']
melted_data = melted_data.sort_values(['Trait', 'Song'])

melted_data.to_csv('data_color.csv', index=False)


In [77]:
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler


def perform_dimensionality_reduction(input_file, output_file):
    # Read data from CSV
    data = pd.read_csv(input_file)
    aggregated_data = data.groupby('Song', as_index=False).agg({'value': 'mean'})
    scaler = MinMaxScaler()
    aggregated_data['value'] = scaler.fit_transform(aggregated_data[['value']])
    categorical_cols = ['Song']
    transformer = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_cols)],
                                    remainder='passthrough')
    # Apply one-hot encoding
    processed_data = transformer.fit_transform(aggregated_data)

    svd = TruncatedSVD(n_components=2)
    svd_result = svd.fit_transform(processed_data)
    result_df = pd.DataFrame(data=svd_result, columns=['Pc1', 'Pc2'])
    result_df.insert(0, 'Song', aggregated_data['Song'])

    # Save the results to output CSV
    result_df.to_csv(output_file, index=False)

In [78]:
input_file = "/content/data_no_color.csv"
output_file = "data_pca.csv"
perform_dimensionality_reduction(input_file, output_file)

In [ ]:
data = pd.read_csv('/content/data_pca.csv')

In [79]:
for index, row in melted_data.iterrows():
    # Check if the 'Trait' column equals 'tempo'
    if row['Trait'] == 'Tempo':
        tempo_value = filtered_data.loc[filtered_data['track_name'] == row['Song'], 'tempo_color'].values[0]
        melted_data.at[index, 'Color'] = tempo_value
with pd.option_context("display.max_rows", None, 'display.max_columns', None):
    display(melted_data)

,Song,Trait,value,Color
114,Adaptation,Acousticness,0.151000,0.151000
115,Blinding Lights,Acousticness,0.001470,0.001470
116,Call Out My Name,Acousticness,0.211000,0.211000
117,Can't Feel My Face,Acousticness,0.112000,0.112000
118,Can't Feel My Face - Martin Garrix Remix,Acousticness,0.007910,0.007910
119,Dark Times,Acousticness,0.114000,0.114000
120,Die For You,Acousticness,0.111000,0.111000
121,Echoes Of Silence,Acousticness,0.951000,0.951000
122,Gone,Acousticness,0.708000,0.708000
123,Heartless,Acousticness,0.006320,0.006320


In [80]:
for index, row in melted_data.iterrows():

    if row['Trait'] == 'Duration (minutes)':
        tempo_value = filtered_data.loc[filtered_data['track_name'] == row['Song'], 'duration_mins_color'].values[0]
        melted_data.at[index, 'Color'] = tempo_value

for index, row in melted_data.iterrows():

    if row['Trait'] == 'Popularity':
        tempo_value = filtered_data.loc[filtered_data['track_name'] == row['Song'], 'track_popularity_color'].values[0]
        melted_data.at[index, 'Color'] = tempo_value

melted_data.to_csv('data_color_final.csv', index=False)

In [85]:
data = pd.read_csv('/content/data_pca.csv')

with pd.option_context("display.max_rows", None, 'display.max_columns', None):
    display(data.dtypes)

print(data['Pc1'].min()-0.1)
print(data['Pc1'].max())
print(data['Pc2'].min())
print(data['Pc2'].max())

Song     object
Pc1     float64
Pc2     float64
dtype: object

-0.10000010661993636
1.0430110873053255
-0.2852449224845849
0.4954620131063652
